In [1]:
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
#!pip install fiona
import fiona

In [2]:
#Clean Air Markets Program Data
campd = pd.read_csv("C:/Users/kromi/OneDrive/Bellevue/Applied Data Science/Weeks 9-12/annual-emissions-facility-aggregation-06534295-07c1-4af7-90b9-ba9a4568f66a.csv")

In [3]:
#Function for reading in multiple files.
def load_data(folder_path, file_keywords=None):
    """
    Loads Excel files in the folder_path that match the provided file_keywords into a dictionary of DataFrames.

    Parameters:
        folder_path (str): The path to the folder with Excel files.
        file_keywords (list of str): Keywords to filter filenames (e.g., ['Schedule_2', 'Plant', 'Multifuel'])

    Returns:
        dict: A dictionary of DataFrames with filenames (no extension) as keys.
    """
    dataframes = {}

    for filename in os.listdir(folder_path):
        if file_keywords is None or any(kw in filename for kw in file_keywords):
            filepath = os.path.join(folder_path, filename)
            name = os.path.splitext(filename)[0]

            try:
                if filename.endswith('.csv'):
                    df = pd.read_csv(filepath, low_memory=False)
                elif filename.endswith(('.xlsx', '.xls')):
                    df = pd.read_excel(filepath, engine='openpyxl')
                else:
                    continue  # skip non-data files
                dataframes[name] = df
            except Exception as e:
                print(f"Failed to load {filename}: {e}")

    return dataframes

## EIA Data

In [5]:
folder = "C:/Users/kromi/OneDrive/Bellevue/Applied Data Science/Weeks 9-12/EIA Data"
file_keywords = ['Schedule_2', 'Schedule_3', 'Schedule_5', 'Plant', 'Generator', 'Multifuel']

eia_data = load_data(folder, file_keywords)
list(eia_data.keys())

['2___Plant_Y2015',
 '2___Plant_Y2016',
 '2___Plant_Y2017',
 '2___Plant_Y2018',
 '2___Plant_Y2019',
 '2___Plant_Y2020',
 '2___Plant_Y2021',
 '2___Plant_Y2022',
 '2___Plant_Y2023',
 '3_1_Generator_Y2015',
 '3_1_Generator_Y2016',
 '3_1_Generator_Y2017',
 '3_1_Generator_Y2018',
 '3_1_Generator_Y2019',
 '3_1_Generator_Y2020',
 '3_1_Generator_Y2021',
 '3_1_Generator_Y2022',
 '3_1_Generator_Y2023',
 '3_4_Multifuel_Y2015',
 '3_5_Multifuel_Y2016',
 '3_5_Multifuel_Y2017',
 '3_5_Multifuel_Y2018',
 '3_5_Multifuel_Y2019',
 '3_5_Multifuel_Y2020',
 '3_5_Multifuel_Y2021',
 '3_5_Multifuel_Y2022',
 '3_5_Multifuel_Y2023']

In [6]:
#Combining DataFrames.
plant_keys = [key for key in eia_data if key.startswith('2___Plant_Y')]
eia_plant_df = pd.concat([eia_data[key].assign(Year=int(key[-4:])) for key in plant_keys], ignore_index=True)

gen_keys = [key for key in eia_data if '3_1_Generator' in key]
eia_gen_df = pd.concat([eia_data[key].assign(Year=int(key[-4:])) for key in gen_keys], ignore_index=True)

## EJScreen Data

In [8]:
folder = "C:/Users/kromi/OneDrive/Bellevue/Applied Data Science/Weeks 9-12/EJScreen Data"
file_keywords = ['EJSCREEN']

ejscreen_files = load_data(folder, file_keywords)
list(ejscreen_files.keys())

Failed to load EJSCREEN_2023_BG_with_AS_CNMI_GU_VI.csv: 'utf-8' codec can't decode byte 0xf1 in position 197766: invalid continuation byte


['EJSCREEN_20150505',
 'EJSCREEN_2017_USPR_Public',
 'EJSCREEN_2019_USPR',
 'EJSCREEN_2020_USPR',
 'EJSCREEN_2021_USPR',
 'EJSCREEN_2022_Full_with_AS_CNMI_GU_VI',
 'EJSCREEN_Full_USPR_2018',
 'EJSCREEN_Full_V3_USPR_TSDFupdate']

In [9]:
ejscreen_files['EJSCREEN_2023_BG_with_AS_CNMI_GU_VI'] = pd.read_csv(
    "C:/Users/kromi/OneDrive/Bellevue/Applied Data Science/Weeks 9-12/EJScreen Data/EJSCREEN_2023_BG_with_AS_CNMI_GU_VI.csv",
    encoding='latin1',
    low_memory=False
)

In [10]:
columns_to_keep = ['FIPS', 'ID', 'PM25', 'OZONE', 'LOWINCPCT', 'MINORPCT']

for key in list(ejscreen_files.keys()):
    try:
        df = ejscreen_files[key].copy()

        # Normalize FIPS/ID before column filtering
        if 'FIPS' in df.columns:
            df['FIPS'] = df['FIPS'].astype(str).str.zfill(12)
        elif 'ID' in df.columns:
            df['FIPS'] = df['ID'].astype(str).str.zfill(12)
        else:
            print(f"⚠️ No FIPS or ID column found in {key}")

        # Keep only relevant columns that exist
        keep_cols = [col for col in ['FIPS', 'PM25', 'OZONE', 'LOWINCPCT', 'MINORPCT'] if col in df.columns]
        df = df[keep_cols]

        # Downcast numeric types
        for col in df.select_dtypes(include='float64').columns:
            df[col] = df[col].astype('float32')
        for col in df.select_dtypes(include='int64').columns:
            df[col] = df[col].astype('int32')

        # Save back
        ejscreen_files[key] = df
        print(f"✅ Cleaned: {key} → {df.shape}")

    except Exception as e:
        print(f"❌ Skipped {key}: {e}")


✅ Cleaned: EJSCREEN_20150505 → (217739, 1)
✅ Cleaned: EJSCREEN_2017_USPR_Public → (220333, 5)
✅ Cleaned: EJSCREEN_2019_USPR → (220333, 5)
✅ Cleaned: EJSCREEN_2020_USPR → (220333, 5)
✅ Cleaned: EJSCREEN_2021_USPR → (220333, 5)
✅ Cleaned: EJSCREEN_2022_Full_with_AS_CNMI_GU_VI → (242940, 5)
✅ Cleaned: EJSCREEN_Full_USPR_2018 → (220333, 5)
✅ Cleaned: EJSCREEN_Full_V3_USPR_TSDFupdate → (220333, 5)
✅ Cleaned: EJSCREEN_2023_BG_with_AS_CNMI_GU_VI → (243021, 4)


In [11]:
pd.concat(ejscreen_files.values())['FIPS'].isna().mean()

0.0

In [12]:
ejscreen_data = pd.concat(ejscreen_files.values(), ignore_index=True)

ejscreen_data.head()

,FIPS,PM25,OZONE,LOWINCPCT,MINORPCT
0,010479573004,NaN,NaN,NaN,NaN
1,010479564002,NaN,NaN,NaN,NaN
2,010479562022,NaN,NaN,NaN,NaN
3,010479569001,NaN,NaN,NaN,NaN
4,010479562012,NaN,NaN,NaN,NaN


In [13]:
ejscreen_data = ejscreen_data.rename(columns={'FIPS': 'GeoID'})

In [14]:
print(ejscreen_data.isna().sum())
print(ejscreen_data.shape)

GeoID             0
PM25         253398
OZONE        253398
LOWINCPCT    219059
MINORPCT     461365
dtype: int64
(2025698, 5)


In [15]:
ejscreen_data = ejscreen_data.dropna(subset = ['PM25', 'OZONE', 'LOWINCPCT', 'MINORPCT'])

In [16]:
print(ejscreen_data.isna().sum())
print(ejscreen_data.shape)

GeoID        0
PM25         0
OZONE        0
LOWINCPCT    0
MINORPCT     0
dtype: int64
(1534346, 5)


# Merging Data
## CAMPD & EIA

In [18]:
print(campd.columns)
print(eia_plant_df.columns)

Index(['State', 'Facility Name', 'Facility ID', 'Year', 'Gross Load (MWh)',
       'Steam Load (1000 lb)', 'SO2 Mass (short tons)',
       'CO2 Mass (short tons)', 'NOx Mass (short tons)', 'Heat Input (mmBtu)'],
      dtype='object')
Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name',
       'Street Address', 'City', 'State', 'Zip', 'County', 'Latitude',
       'Longitude', 'NERC Region', 'Balancing Authority Code',
       'Balancing Authority Name', 'Name of Water Source',
       'Primary Purpose (NAICS Code)', 'Regulatory Status', 'Sector',
       'Sector Name',
       'Net Metering (for facilities with solar or wind generation)',
       'FERC Cogeneration Status', 'FERC Cogeneration Docket Number',
       'FERC Small Power Producer Status',
       'FERC Small Power Producer Docket Number',
       'FERC Exempt Wholesale Generator Status',
       'FERC Exempt Wholesale Generator Docket Number', 'Ash Impoundment?',
       'Ash Impoundment Lined?', 'Ash Impoundment Status',

In [19]:
df = campd.merge(
    eia_plant_df,
    left_on='Facility ID',     
    right_on='Plant Code',    
    how='left'
)

df.head()

,State_x,Facility Name,Facility ID,Year_x,Gross Load (MWh),Steam Load (1000 lb),SO2 Mass (short tons),CO2 Mass (short tons),NOx Mass (short tons),Heat Input (mmBtu),...,Natural Gas Pipeline Name,Year_y,Energy Storage,Natural Gas LDC Name,Natural Gas Pipeline Name 1,Natural Gas Pipeline Name 2,Natural Gas Pipeline Name 3,Pipeline Notes,Natural Gas Storage,Liquefied Natural Gas Storage
0,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,Bay Gas,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,NaN,2016.0,N,NaN,BAY GAS STORAGE,NaN,NaN,NaN,N,X
2,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,NaN,2017.0,N,NaN,BAY GAS STORAGE,NaN,NaN,NaN,N,X
3,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,NaN,2018.0,N,NaN,BAY GAS STORAGE,NaN,NaN,NaN,N,X
4,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,NaN,2019.0,N,NaN,BAY GAS STORAGE,NaN,NaN,NaN,N,X


In [20]:
#Dropping redundant Year_y column
df.drop(columns='Year_y', inplace=True, errors='ignore')

df.rename(columns={'Year_x': 'Year'}, inplace=True)

In [21]:
#Dropping redundant State_y column
df.drop(columns='State_y', inplace=True, errors='ignore')

# Rename Year_x to Year
df.rename(columns={'State_x': 'State'}, inplace=True)

In [22]:
eia_gen_df.columns

Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State',
       'County', 'Generator ID', 'Technology', 'Prime Mover', 'Unit Code',
       'Ownership', 'Duct Burners',
       'Can Bypass Heat Recovery Steam Generator?',
       'RTO/ISO LMP Node Designation',
       'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
       'Nameplate Capacity (MW)', 'Nameplate Power Factor',
       'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Minimum Load (MW)',
       'Uprate or Derate Completed During Year',
       'Month Uprate or Derate Completed', 'Year Uprate or Derate Completed',
       'Status', 'Synchronized to Transmission Grid', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Associated with Combined Heat and Power System', 'Sector Name',
       'Sector', 'Topping or Bottoming', 'Energy Source 1', 'Energy Source 2',
       'Energy Source 3', 'Energy Source 4', 'Energy Source 5',
       'Ene

In [23]:
df = df.merge(
    eia_gen_df,
    left_on=['Facility ID', 'Year'],       
    right_on=['Plant Code', 'Year'],         
    how='left'
)

df.head()

,State_x,Facility Name,Facility ID,Year,Gross Load (MWh),Steam Load (1000 lb),SO2 Mass (short tons),CO2 Mass (short tons),NOx Mass (short tons),Heat Input (mmBtu),...,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Cofire Fuels?,Switch Between Oil and Natural Gas?,Turbines or Hydrokinetic Buoys,Multiple Fuels?
0,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
1,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
2,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
3,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
4,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN


In [24]:
#Dropping redundant State_y column
df.drop(columns='State_y', inplace=True, errors='ignore')

df.rename(columns={'State_x': 'State'}, inplace=True)

df.head()

,State,Facility Name,Facility ID,Year,Gross Load (MWh),Steam Load (1000 lb),SO2 Mass (short tons),CO2 Mass (short tons),NOx Mass (short tons),Heat Input (mmBtu),...,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Cofire Fuels?,Switch Between Oil and Natural Gas?,Turbines or Hydrokinetic Buoys,Multiple Fuels?
0,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
1,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
2,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
3,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN
4,AL,Barry,3,2015,11948203.0,NaN,8688.188,7969417.474,5661.177,9.841812e+07,...,,,,NaN,,,NaN,NaN,NaN,NaN


In [25]:
columns_to_keep = ['State', 'Facility Name', 'Facility ID', 'Year', 'SO2 Mass (short tons)', 'NOx Mass (short tons)', 'CO2 Mass (short tons)',
                   'Heat Input (mmBtu)', 'Gross Load (MWh)', 'Latitude', 'Longitude']

df = df[columns_to_keep].dropna(subset=['Latitude', 'Longitude'])

df.shape

(404841, 11)

## CAMPD/EIA & EJScreen

In [27]:
campd_geo = df.dropna(subset=['Latitude', 'Longitude']).copy()
campd_geo['geometry'] = [Point(xy) for xy in zip(campd_geo['Longitude'], campd_geo['Latitude'])]
campd_geo = gpd.GeoDataFrame(campd_geo, geometry='geometry', crs='EPSG:4326')

In [28]:
ejscreen_data.columns.tolist()

['GeoID', 'PM25', 'OZONE', 'LOWINCPCT', 'MINORPCT']

In [29]:
# Path to the .gdb folder
gdb_path = r"C:/Users/kromi/OneDrive/Bellevue/Applied Data Science/Weeks 9-12/EJScreen Data/EJScreen_2024_BG_StatePct_with_AS_CNMI_GU_VI.gdb"

fiona.listlayers(gdb_path)

['EJSCREEN_StatePct_with_AS_CNMI_GU_VI', 'States']

In [30]:
layer_name = "EJSCREEN_StatePct_with_AS_CNMI_GU_VI"

gdf = gpd.read_file(gdb_path, layer=layer_name)
print(gdf.columns.tolist())

['ID', 'STATE_NAME', 'ST_ABBREV', 'CNTY_NAME', 'REGION', 'ACSTOTPOP', 'ACSIPOVBAS', 'ACSEDUCBAS', 'ACSTOTHH', 'ACSTOTHU', 'ACSUNEMPBAS', 'ACSDISABBAS', 'DEMOGIDX_2', 'DEMOGIDX_5', 'PEOPCOLOR', 'PEOPCOLORPCT', 'LOWINCOME', 'LOWINCPCT', 'UNEMPLOYED', 'UNEMPPCT', 'DISABILITY', 'DISABILITYPCT', 'LINGISO', 'LINGISOPCT', 'LESSHS', 'LESSHSPCT', 'UNDER5', 'UNDER5PCT', 'OVER64', 'OVER64PCT', 'LIFEEXPPCT', 'PM25', 'OZONE', 'DSLPM', 'RSEI_AIR', 'PTRAF', 'PRE1960', 'PRE1960PCT', 'PNPL', 'PRMP', 'PTSDF', 'UST', 'PWDIS', 'NO2', 'DWATER', 'D2_PM25', 'D5_PM25', 'D2_OZONE', 'D5_OZONE', 'D2_DSLPM', 'D5_DSLPM', 'D2_RSEI_AIR', 'D5_RSEI_AIR', 'D2_PTRAF', 'D5_PTRAF', 'D2_LDPNT', 'D5_LDPNT', 'D2_PNPL', 'D5_PNPL', 'D2_PRMP', 'D5_PRMP', 'D2_PTSDF', 'D5_PTSDF', 'D2_UST', 'D5_UST', 'D2_PWDIS', 'D5_PWDIS', 'D2_NO2', 'D5_NO2', 'D2_DWATER', 'D5_DWATER', 'P_DEMOGIDX_2', 'P_DEMOGIDX_5', 'P_PEOPCOLORPCT', 'P_LOWINCPCT', 'P_UNEMPPCT', 'P_DISABILITYPCT', 'P_LINGISOPCT', 'P_LESSHSPCT', 'P_UNDER5PCT', 'P_OVER64PCT', 'P_LI

In [31]:
ej_geo = gpd.read_file(gdb_path, layer=layer_name)[['ID', 'geometry']]
ej_geo = ej_geo.rename(columns={'ID': 'GeoID'})

In [32]:
campd_geo = campd_geo.to_crs(ej_geo.crs)

In [33]:
campd_geo.columns.tolist()

['State',
 'Facility Name',
 'Facility ID',
 'Year',
 'SO2 Mass (short tons)',
 'NOx Mass (short tons)',
 'CO2 Mass (short tons)',
 'Heat Input (mmBtu)',
 'Gross Load (MWh)',
 'Latitude',
 'Longitude',
 'geometry']

In [34]:
df = gpd.sjoin(campd_geo, ej_geo, how='left', predicate='intersects')

In [35]:
print("df['GeoID'] sample:")
print(df['GeoID'].dropna().unique()[:5])

print("\nejscreen_data['GeoID'] sample:")
print(ejscreen_data['GeoID'].dropna().unique()[:5])

df['GeoID'] sample:
['010970058001' '010550013001' '011270215004' '481410035012'
 '010630602004']

ejscreen_data['GeoID'] sample:
['010010201001' '010010201002' '010010202001' '010010202002'
 '010010203001']


In [36]:
df = df.merge(ejscreen_data, on='GeoID', how='left')

df.head()

,State,Facility Name,Facility ID,Year,SO2 Mass (short tons),NOx Mass (short tons),CO2 Mass (short tons),Heat Input (mmBtu),Gross Load (MWh),Latitude,Longitude,geometry,index_right,GeoID,PM25,OZONE,LOWINCPCT,MINORPCT
0,AL,Barry,3,2015,8688.188,5661.177,7969417.474,9.841812e+07,11948203.0,31.0069,-88.0103,POINT (-88.0103 31.0069),2782,010970058001,9.035454,35.013016,0.481414,0.427924
1,AL,Barry,3,2015,8688.188,5661.177,7969417.474,9.841812e+07,11948203.0,31.0069,-88.0103,POINT (-88.0103 31.0069),2782,010970058001,8.854428,35.954987,0.329457,0.361191
2,AL,Barry,3,2015,8688.188,5661.177,7969417.474,9.841812e+07,11948203.0,31.0069,-88.0103,POINT (-88.0103 31.0069),2782,010970058001,8.762251,35.371883,0.379430,0.366209
3,AL,Barry,3,2015,8688.188,5661.177,7969417.474,9.841812e+07,11948203.0,31.0069,-88.0103,POINT (-88.0103 31.0069),2782,010970058001,8.870061,36.325394,0.374755,0.401565
4,AL,Barry,3,2015,8688.188,5661.177,7969417.474,9.841812e+07,11948203.0,31.0069,-88.0103,POINT (-88.0103 31.0069),2782,010970058001,8.870061,36.325394,0.206022,0.409422


# Data Cleaning

In [38]:
df.columns.tolist()

['State',
 'Facility Name',
 'Facility ID',
 'Year',
 'SO2 Mass (short tons)',
 'NOx Mass (short tons)',
 'CO2 Mass (short tons)',
 'Heat Input (mmBtu)',
 'Gross Load (MWh)',
 'Latitude',
 'Longitude',
 'geometry',
 'index_right',
 'GeoID',
 'PM25',
 'OZONE',
 'LOWINCPCT',
 'MINORPCT']

In [40]:
#Specifying columns to keep.
columns_to_keep = ['Facility Name', 'Facility ID', 'Year', 'Latitude', 'Longitude', 'GeoID', 'SO2 Mass (short tons)', 'NOx Mass (short tons)',
                   'CO2 Mass (short tons)', 'Heat Input (mmBtu)', 'Gross Load (MWh)', 'PM25', 'OZONE', 'LOWINCPCT', 'MINORPCT', 'State']

df = df[columns_to_keep].copy()

In [42]:
df.isna().sum()

Facility Name                 0
Facility ID                   0
Year                          0
Latitude                      0
Longitude                     0
GeoID                         0
SO2 Mass (short tons)    138590
NOx Mass (short tons)     24144
CO2 Mass (short tons)    193400
Heat Input (mmBtu)        24963
Gross Load (MWh)         134966
PM25                       5599
OZONE                      5599
LOWINCPCT                  5599
MINORPCT                   5599
State                         0
dtype: int64

In [44]:
df = df.dropna(subset = ['SO2 Mass (short tons)', 'NOx Mass (short tons)', 'CO2 Mass (short tons)', 'Heat Input (mmBtu)', 'Gross Load (MWh)', 'PM25',
                         'OZONE', 'LOWINCPCT', 'MINORPCT'])

df.shape

(1884379, 16)

In [46]:
#Renaming columns
rename_dict = {
    'OZONE': 'Ozone',
    'LOWINCPCT': 'Low Income (pct)',
    'MINORPCT': 'Minority Pop (pct)'
}

df = df.rename(columns=rename_dict)

In [48]:
df.columns.tolist()

['Facility Name',
 'Facility ID',
 'Year',
 'Latitude',
 'Longitude',
 'GeoID',
 'SO2 Mass (short tons)',
 'NOx Mass (short tons)',
 'CO2 Mass (short tons)',
 'Heat Input (mmBtu)',
 'Gross Load (MWh)',
 'PM25',
 'Ozone',
 'Low Income (pct)',
 'Minority Pop (pct)',
 'State']

In [50]:
df.to_csv("EJ Data.csv", index = False)